### TAREA BLOQUE 1 - INTRODUCCIÓN

Autor: Iñaki Gangutia Arrázola

### FECHA LIMITE SABADO 8 NOVIEMBRE

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Crea un programa extractor cuya función sea obtener datos desde varias fuentes de datos online (APIs).
* Este programa puede contener múltiples opciones, pero es necesario que existan métodos para descargar información histórica de precios tanto de acciones como de índices.
* Independientemente de la fuente de información, el formato de salida de esta información debe ser estandarizado. Es decir, con independencia de la fuente original, nuestro programa debe darnos en salida objetos iguales. Por ejemplo: independientemente de la api utilizada, el formato de precio histórico debe ser compatible con el formato de salida que resulte de utilizar otra API diferente.
* Añade opción de conseguir otra tipología de datos a tu gusto.
* Haz que el extractor pueda conseguir N series de datos al mismo tiempo dado un input que induzca a ello.

Definimos conjuntos de activos divididos por tipos de activo o región, a modo de facilitar la recolección de información de una forma más sencilla y accesible.

In [169]:
def API_data(TKRs, start, end):
    import numpy as np
    import pandas as pd
    import pandas_datareader.data as pdr
    from ecbdata import ecbdata as ecb

    '''
    Retrieve financial time series data for various asset classes based on the specified ticker group and date range

    :param TKR: Asset group or ticker symbol(s) to fetch data for
    :param start: Start date for the data retrieval period
    :param end: End date for the data retrieval period
    :return: DataFrame or Series containing the requested financial data
    :rtype: Series[Any] | Any | DataFrame
    '''


    # Pandas Data Reader API function
    def API_pdr(tickers, source, start, end):
        data = pdr.DataReader(
            tickers,
            source,
            start=start,
            end=end
            )
        data.index = pd.to_datetime(data.index)
        return data
    

    # European Central Bank API function
    def API_ecb(tickers, start, end):
        data = pd.DataFrame()

        for tick in tickers:
            data_n = ecb.get_series(
                tick,
                start=start,
                end=end
                )

            data_n = data_n[['TIME_PERIOD', 'OBS_VALUE']].set_index('TIME_PERIOD')
            data_n.index = pd.to_datetime(data_n.index)
            data[tick] = data_n

        data.rename(columns={'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_3M': 'EU3M',
                            'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_1Y': 'EU1Y',
                            'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_2Y': 'EU2Y',
                            'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_5Y': 'EU5Y',
                            'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_10Y': 'EU10Y',
                            'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_20Y': 'EU20Y',
                            'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_30Y': 'EU30Y'
                            }, 
                            inplace=True)
        return data


    if type(TKRs) is str:
        TKRs = [TKRs]

    elif type(TKRs) is list:
        pass

    else: 
        raise TypeError('TKRs must be a string or a list of strings.')


    data = pd.DataFrame(np.nan, index=pd.date_range(start=start, end=end), columns=[])

    for TKR in TKRs:

        # World Indexes
        if TKR == 'WI':
            tickers = ['^SPX',  # S&P 500
                       '^NDX',  # Nasdaq 100
                       '^DJC',  # Dow Jones Industrial Average
                       '^FTM',   # UK
                       '^DAX',  # Germany
                       '^CAC'  # France
                       ]
            data_n = API_pdr(tickers, 'stooq', start, end)['Close']
        
        # Single Stocks (World top 45 by revenue, excluding China, Hong Kong, South Korea and Saudi Arabia)
        elif TKR == 'SS':
            tickers = ['BRK-B', 'AAPL', 'GOOG',
                       'MSFT', 'JPM', 'NVDA',
                       'META', 'AMZN', 'XOM',
                       'TM', 'TSM', 'HSBC',
                       'EQNR', 'TTE', 'WMT',
                       'SHEL', 'TCEHY', 'CVX',
                       'BAC', 'V', 'CDI.PA',
                       'VZ', 'BABA', 'BN',
                       'HD', 'WFC', 'FNMA',
                       'SAN', 'PG', 'UNH',
                       'VOW3.DE', 'PBR', 'NVO'
                       ]
            data_n = API_pdr(tickers, 'stooq', start, end)['Close']

        # EUrope Government Bond rate (all rating issuers)
        elif TKR == 'EUGB':
            tickers = ['YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_3M',  # 3m
                       'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_1Y',  # 1y
                       'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_2Y',  # 2y
                       'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_5Y',  # 5y
                       'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_10Y',  # 10y
                       'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_20Y',  # 20y
                       'YC.B.U2.EUR.4F.G_N_C.SV_C_YM.SR_30Y'   # 30y
                       ]
            data_n = API_ecb(tickers, start, end)

        # U.S. Government Bond rate (Treasury yields)
        elif TKR == 'USGB':
            tickers = ['DGS3MO',  # 3m
                       'DGS1',  # 1y
                       'DGS2',  # 2y
                       'DGS5',  # 5y
                       'DGS10',  # 10y
                       'DGS20',  # 20y
                       'DGS30'  # 30y
                       ]
            data_n = API_pdr(tickers, 'fred', start, end)

        # Commodities
        elif TKR == 'COMM':
            tickers = ['CL.F',  # Oil (WTI)
                       'CB.F',  # Oil (Brent)
                       'NG.F',  # Natural Gas
                       'GC.F',  # Gold
                       'SI.F',  # Silver
                       'HG.F',  # Copper
                       'PL.F',  # Platinum
                       'ZS.F',  # Soybeans
                       'KE.F',  # Wheat
                       'ZC.F',  # Corn
                       'SB.F',  # Sugar
                       'CT.F'  # Cotton
                       ]
            data_n = API_pdr(tickers, 'stooq', start, end)['Close']

        
        # Foreign Exchange
        elif TKR == 'FX':
            tickers = ['EURUSD',  # Euro-Dollar
                       'USDJPY',  # USD/JPY (Yen Japonés)
                       'GBPUSD',  # Libra Esterlina
                       'USDCAD',  # Canadian Dollar
                       'AUDUSD',  # Australian Dollar
                       'EURCHF',  # Swiss Franc
                       'NZDUSD'  # New Zealand Dollar (Kiwi)
                       ]
            data_n = API_pdr(tickers, 'stooq', start, end)['Close']
            
        else:
            tickers = TKR
            data_n = API_pdr(tickers, 'stooq', start, end)['Close']
        
        data = data.merge(data_n, left_index=True, right_index=True, how='outer')
        data.index.name = 'Date'


    return data

In [170]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
from ecbdata import ecbdata as ecb


pdr.DataReader(
    'EURUSD',
    'stooq',
    start=start,
    end=end
    )

""
Exceeded the daily hits limit


In [ ]:
start = '2025-01-01'
end = '2025-01-06'

API_data(['FX', 'EUGB', 'COMM'], start, end)

KeyError: 'Close'

Hagamos que los datos sean coherentes. Cada serie de datos debe ser un objeto. Crea DataClasses para las series de precios. Existiendo estos objetos, ¿qué es una cartera?

Añade métodos a las dataclasses de series de precios que incorporen información estadística relevante. Haz que los métodos para la información más básica (media y desviación típica) se apliquen automáticamente.


Dado un valor o una cartera, elabora un pequeño programa que genere una simulación de Monte Carlo para su evolución.
* Intenta que esta simulación sea maleable por el usuario por parámetros.
* Haz que la simulación pueda ser tanto de la cartera en su conjunto como de los elementos que la componen.

Convierte el proceso de Monte Carlo en un método para la DataClass de la cartera. Incluye otro método que muestre por pantalla visualmente el resultado.

Incluye métodos para la “limpieza” y el preprocesado de los datos. ¿Debería el programa aceptarte cualquier tipo de input siempre que exista una serie temporal de precios?


Dada una cartera, crea un método .report (con los parámetros que consideres oportunos) que genere texto formateado en markdown y pueda mostrar por pantalla aquel análisis que consideres relevante, así como advertencias y cualquier otro aspecto.

Antiguamente, generar gráficos verdaderamente resultones era tedioso y no tenia gran aporte intelectual; la IA facilita enormemente todo esto. Intenta pensar en el tipo de visualizaciones más útiles para un usuario con su ayuda y guárdalas para que un método .plots_report() las muestre.

Estás construyendo un programa que empieza a tener dependencias, jerarquías, clases dentro de clases… es importante mantener la perspectiva general de lo que estás construyendo. Muchas veces, de hecho, igual debería ser tu primer paso. Intenta plasmar esta estructura utilizando algún recurso para mostrar diagramas. Esta es sencilla y útil https://github.com/stan-smith/FossFLOW 